In [3]:
import os
import dotenv
import openai

dotenv.load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

langchain_hub_api_key = os.getenv('LANGCHAIN_HUB_API_KEY')

from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain.prompts import PromptTemplate

def make_tool_from_DocRetriever(
        doc_path, 
        name: str, 
        description: str,
        chunk_size=500,
        chunk_overlap=30,
        embedding=OpenAIEmbeddings()):

    from langchain_community.document_loaders import TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.vectorstores import FAISS
    from langchain.tools.retriever import create_retriever_tool

    data = TextLoader(doc_path, encoding='utf-8').load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    splits = text_splitter.split_documents(data)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
    retriever = vectorstore.as_retriever(search_type="similarity")
    retriever_tool = create_retriever_tool(
        retriever,
        name=name,
        description=description,
        )
    return retriever_tool

qna_tool = make_tool_from_DocRetriever(
    doc_path='D:/SandBox/Dev/ui/chatbot/assets/qna.txt',
    name='qna-relevant_statutory_provisions-tool',
    description='민원 질문에 대한 해결방법과 근거법령을 제시해야할 때 유용합니다.',
    )

links_tool = make_tool_from_DocRetriever(
    doc_path='D:/SandBox/Dev/ui/chatbot/assets/link.txt',
    name='statute_links-tool',
    description='근거법령의 link를 제시해야할 때 유용합니다.',
    )

# prompt = hub.pull("hwchase17/react", api_key=langchain_hub_api_key)
prompt = PromptTemplate(
    input_variables=["agent_scratchpad", "input", "tool_names", "tools"],
    template='''
    다음 질문에 최선을 다해 답변하세요. 당신은 다음 도구들을 사용할 수 있습니다:

    {tools}

    다음 형식을 사용하세요:

    질문: 당신이 답변해야 할 입력 질문
    생각: 무엇을 해야 할지 항상 생각하세요
    행동: 취할 행동을 선택하세요 [{tool_names}] 중 하나
    행동 입력: 행동에 대한 입력값
    관찰: 행동의 결과
    ... (이 생각/행동/행동 입력/관찰이 N번 반복될 수 있습니다)
    생각: 이제 최종 답을 알았습니다
    최종 답변: 원래 질문에 대한 최종 답변
    
    시작하세요!

    질문: {input}
    생각: {agent_scratchpad}
    '''
)
tools = [qna_tool, links_tool]
llm = ChatOpenAI(model='gpt-4o', temperature=0)

agent = create_react_agent(llm, tools, prompt)
executor = AgentExecutor(
    agent=agent,
    tools=tools, 
    handle_parsing_errors=True,
    max_iterations=5,
    max_execution_time=10,
    verbose=True, 
    return_intermediate_steps=True
    )


def get_answer(input):
    result = executor.invoke({"input":input})
    if 'Agent stopped' in result['output']:
        return result['intermediate_steps'][-1]
    else:
        return result['output']
    return result

In [ ]:
# question = "여행할 수 없는 나라를 가고 싶은 경우 어떻게 해야해? 근거 법령에 대한 링크도 알려줘."
question = "주민등록본을 발급받으려고 했는데 내 주민등록번호가 말소됐대 이거 어떻게 해야해? 근거 법령에 대한 링크도 알려줘."

get_answer(question)